# Combinando Datasets: Merge y Join

Una característica esencial que ofrece Pandas son sus operaciones join y merge de alto rendimiento en memoria.
Si alguna vez has trabajado con bases de datos, deberías estar familiarizado con este tipo de interacción de datos.
La interfaz principal para esto es la función ``pd.merge``, y veremos algunos ejemplos de cómo puede funcionar en la práctica.


In [2]:
import pandas as pd
import numpy as np
from utils.funciones import Display

## Álgebra relacional

El comportamiento implementado en ``pd.merge()`` es un subconjunto de lo que se conoce como *álgebra relacional*, que es un conjunto formal de reglas para manipular datos relacionales, y forma la base conceptual de las operaciones disponibles en la mayoría de las bases de datos.
El punto fuerte del enfoque del álgebra relacional es que propone varias operaciones primitivas, que se convierten en los bloques de construcción de operaciones más complicadas sobre cualquier conjunto de datos.
Con este léxico de operaciones fundamentales implementado de forma eficiente en una base de datos u otro programa, se puede realizar una amplia gama de operaciones compuestas bastante complicadas.

Pandas implementa varios de estos bloques de construcción fundamentales en la función ``pd.merge()`` y el método relacionado ``join()`` de ``Series`` y ``Dataframe``.


## Categorías de Joins

La función ``pd.merge()`` implementa varios tipos de uniones: las uniones *uno-a-uno*, *muchos-a-uno* y *muchos-a-muchos*.
A los tres tipos de uniones se accede mediante una llamada idéntica a la interfaz ``pd.merge()``; el tipo de unión realizada depende de la forma de los datos de entrada.
Aquí mostraremos ejemplos sencillos de los tres tipos de uniones, y discutiremos las opciones detalladas más adelante.

### Joins individuales

Tal vez el tipo más sencillo de expresión de fusión sea la unión uno a uno, que en muchos aspectos es muy similar a la concatenación por columnas.

Como ejemplo concreto, consideremos los dos ``DataFrame`` siguientes, que contienen información sobre varios empleados de una empresa:

In [3]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})

Display('df1', 'df2', context=locals())

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


Para combinar esta información en un único ``DataFrame``, podemos utilizar la función ``pd.merge()``:

In [4]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


La función ``pd.merge()`` reconoce que cada ``DataFrame`` tiene una columna "empleado", y une automáticamente utilizando esta columna como clave.
El resultado de la fusión es un nuevo ``DataFrame`` que combina la información de las dos entradas.
Observa que el orden de las entradas en cada columna no se mantiene necesariamente: en este caso, el orden de la columna "empleado" difiere entre ``df1`` y ``df2``, y la función ``pd.merge()`` lo tiene en cuenta correctamente.
Además, hay que tener en cuenta que la fusión en general descarta el índice, excepto en el caso especial de las fusiones por índice (las palabras clave ``left_index`` y ``right_index``, de las que hablaremos más adelante).

### Joins muchos-a-uno

Los joins muchos-a-uno son uniones en las que una de las dos columnas clave contiene entradas duplicadas.
Para el caso muchos-a-uno, el ``DataFrame`` resultante conservará esas entradas duplicadas según corresponda.

In [5]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Alex', 'Steve']})
Display('df3', 'df4', 'pd.merge(df3, df4)', context=locals())

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering       Alex
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012       Alex
2     Lisa  Engineering       2004       Alex
3      Sue           HR       2014      Steve

El ``DataFrame`` resultante tiene una columna adicional con la información del "supervisor", donde la información se repite en uno o más lugares según lo requieran las entradas.

### Joins muchos-a-muchos

Los joins muchos-a-muchos son un poco confusas conceptualmente, pero sin embargo están bien definidas.
Si la columna clave tanto de la matriz izquierda como de la derecha contiene duplicados, el resultado es una fusión de muchos a muchos.
Quizá esto quede más claro con un ejemplo concreto.
Considere lo siguiente, donde tenemos un ``DataFrame`` que muestra una o más habilidades asociadas con un grupo en particular.
Realizando una unión de muchos a muchos, podemos recuperar las habilidades asociadas a cualquier persona individual:

In [6]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
Display('df1', 'df5', "pd.merge(df1, df5)", context=locals())

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


Estos tres tipos de uniones se pueden utilizar con otras herramientas de Pandas para implementar una amplia gama de funcionalidades.
Pero en la práctica, los conjuntos de datos raramente están tan limpios como con el que estamos trabajando aquí.

## Especificación de la clave de fusión

Ya hemos visto el comportamiento por defecto de ``pd.merge()``: busca uno o más nombres de columna que coincidan entre las dos entradas y los utiliza como clave.
Sin embargo, a menudo los nombres de las columnas no coinciden tan bien, y ``pd.merge()`` proporciona una variedad de opciones para manejar esto.

### La palabra clave ``on``

Lo más sencillo es especificar explícitamente el nombre de la columna clave mediante la palabra clave ``on``, que recibe un nombre de columna o una lista de nombres de columna:

In [7]:
Display('df1', 'df2', "pd.merge(df1, df2, on='employee')", context=locals())

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

Esta opción sólo funciona si tanto el ``DataFrame`` izquierdo como el derecho tienen el nombre de columna especificado.

### Las palabras clave ``left_on`` y ``right_on``

En ocasiones, es posible que desee combinar dos conjuntos de datos con nombres de columna diferentes; por ejemplo, podemos tener un conjunto de datos en el que el nombre del empleado esté etiquetado como "nombre" en lugar de "empleado".
En este caso, podemos utilizar las palabras clave ``left_on`` y ``right_on`` para especificar los dos nombres de columna:

In [8]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
Display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")', context=locals())

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

El resultado tiene una columna redundante que podemos eliminar si lo deseamos, por ejemplo, utilizando el método ``drop()`` de ``DataFrame``:

In [9]:
# pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)
pd.merge(df1, df3, left_on="employee", right_on="name").drop(columns='name')

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### Las palabras clave ``left_index`` y ``right_index``

A veces, en lugar de fusionar en una columna, se desea hacerlo en un índice.
Por ejemplo, tus datos podrían tener este aspecto:

In [10]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
Display('df1a', 'df2a', context=locals())

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Puedes utilizar el índice como clave para la fusión especificando los indicadores ``left_index`` y/o ``right_index`` en ``pd.merge()``:

In [11]:
Display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)", context=locals())

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Por comodidad, los ``DataFrame`` implementan el método ``join()``, que realiza una fusión que por defecto une los índices:

In [12]:
Display('df1a', 'df2a', 'df1a.join(df2a)', context=locals())

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Si quieres mezclar índices y columnas, puedes combinar ``left_index`` con ``right_on`` o ``left_on`` con ``right_index`` para obtener el comportamiento deseado:

In [13]:
Display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')", context=locals())

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

Todas estas opciones también funcionan con múltiples índices y/o múltiples columnas; la interfaz para este comportamiento es muy intuitiva.
Para más información sobre esto, consulte la sección ["Merge, Join, and Concatenate"](http://pandas.pydata.org/pandas-docs/stable/merging.html) de la documentación de Pandas.

## Especificación de la aritmética de conjuntos para joins

En todos los ejemplos anteriores hemos pasado por alto una consideración importante a la hora de realizar una unión: el tipo de aritmética de conjuntos utilizado en la unión.
Esto ocurre cuando un valor aparece en una columna clave pero no en la otra. Veamos este ejemplo:

In [14]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
Display('df6', 'df7', 'pd.merge(df6, df7)', context=locals())

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

Aquí hemos fusionado dos conjuntos de datos que sólo tienen una entrada de "name" en común: María.
Por defecto, el resultado contiene la *intersección* de los dos conjuntos de entradas; esto es lo que se conoce como *inner join*.
Podemos especificarlo explícitamente mediante la palabra clave ``how``, que por defecto es ``"inner"``:

In [15]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Otras opciones para la palabra clave ``how`` son ``'outer``, ``'left`` y ``'right``.
Un *outer join* devuelve un join sobre la unión de las columnas de entrada, y rellena todos los valores perdidos con NAs:

In [16]:
Display('df6', 'df7', "pd.merge(df6, df7, how='outer')", context=locals())

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0  Joseph    NaN  beer
1    Mary  bread  wine
2    Paul  beans   NaN
3   Peter   fish   NaN

Las opciones *left join* y *right join* devuelven uniones sobre las entradas de la izquierda y la derecha, respectivamente.
Por ejemplo:

In [17]:
Display('df6', 'df7', "pd.merge(df6, df7, how='left')", context=locals())

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

Las filas de salida corresponden ahora a las entradas de la entrada izquierda. Utilizando
``how='right'`` funciona de forma similar.

## Solapamiento de nombres de columna: La palabra clave ``suffixes``

Por último, puede darse el caso de que sus dos ``DataFrame`` de entrada tengan nombres de columna contradictorios.
Considere este ejemplo:

In [18]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
Display('df8', 'df9', 'pd.merge(df8, df9, on="name")', context=locals())

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

Dado que la salida tendría dos nombres de columna en conflicto, la función de fusión añade automáticamente un sufijo ``_x`` o ``_y`` para hacer que las columnas de salida sean únicas.
Si estos valores por defecto no son apropiados, es posible especificar un sufijo personalizado utilizando la palabra clave ``suffixes``:

In [19]:
Display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])', context=locals())

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

agv_Estos sufijos nos podrian valer par eliminar alguna de las columnas si ya no la necesitamos

In [20]:
#tomo el dataFrame en una variable para operar sobre ella
aa = pd.merge (df8, df9, on="name", suffixes=["_L", "_R"])


In [21]:
#aquí opero sobre las columnas i, para quitar la que tiene el sufijo
for i in aa.columns:
    if '_L' in i:
        aa = aa.drop(columns=[i])

aa


,name,rank_R
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


Estos sufijos funcionan en cualquiera de los posibles patrones de unión, y funcionan también si hay varias columnas superpuestas.

## Ejemplo: Datos de los Estados de EE.UU.

Las operaciones de fusión y unión son más frecuentes cuando se combinan datos de distintas fuentes.
Aquí veremos un ejemplo de datos sobre los estados de EE.UU. y su población.
Los archivos de datos se encuentran en http://github.com/jakevdp/data-USstates/:

In [22]:
# # Descargar los archivos dentro de la carpeta data
!curl -o data/state-population.csv https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
!curl -o data/state-areas.csv https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
!curl -o data/state-abbrevs.csv https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57935  100 57935    0     0   204k      0 --:--:-- --:--:-- --:--:--  205k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   835  100   835    0     0   2572      0 --:--:-- --:--:-- --:--:--  2569
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   872  100   872    0     0   2655      0 --:--:-- --:--:-- --:--:--  2658


Echemos un vistazo a los tres conjuntos de datos, utilizando la función Pandas ``read_csv()``:

In [23]:
pop = pd.read_csv('data/state-population.csv')
#pop = pd.read_csv(https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv)
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

Display('pop.head()', 'areas.head()', 'abbrevs.head()', context=locals())

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

Con esta información, supongamos que queremos calcular un resultado relativamente sencillo: clasificar los estados y territorios de EE.UU. según su densidad de población en 2010.
Es evidente que disponemos de los datos necesarios para obtener este resultado, pero tendremos que combinar los conjuntos de datos para obtenerlo.

Empezaremos con una combinación múltiple que nos dará el nombre completo del estado dentro del ``DataFrame`` de población.
Queremos hacer la combinación basándonos en la columna "estado/región" de "pop" y en la columna "abreviatura" de "abbrevs".
Utilizaremos ``how='outer'`` para asegurarnos de que no se pierde ningún dato por no coincidir las etiquetas.

In [30]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop(columns='abbreviation') # eliminar información duplicada
#merged.head() #solo veo las 5 primeras filas, por defecto
#merged.head(9)  #veo las filas entre parentesis
merged   # veo todo el código

,state/region,ages,year,population,state
0,AK,total,1990,553290.0,Alaska
1,AK,under18,1990,177502.0,Alaska
2,AK,total,1992,588736.0,Alaska
3,AK,under18,1991,182180.0,Alaska
4,AK,under18,1992,184878.0,Alaska
...,...,...,...,...,...
2539,WY,under18,1993,137458.0,Wyoming
2540,WY,total,1991,459260.0,Wyoming
2541,WY,under18,1991,136720.0,Wyoming
2542,WY,under18,1990,136078.0,Wyoming


In [31]:
merged.shape

(2544, 5)

Comprobemos si se ha producido alguna discordancia, lo que podemos hacer buscando filas con nulos:

In [25]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Algunos de los datos de ``población`` son nulos; ¡averigüemos cuáles son!

In [34]:
merged[merged['population'].isnull()].head() #pongo head para ver los 5 primeros, y de qué va el problema. 

,state/region,ages,year,population,state
1872,PR,under18,1990,NaN,NaN
1873,PR,total,1990,NaN,NaN
1874,PR,total,1991,NaN,NaN
1875,PR,under18,1991,NaN,NaN
1876,PR,total,1993,NaN,NaN


Parece que todos los valores nulos de población son de Puerto Rico anteriores al año 2000; esto se debe probablemente a que estos datos no están disponibles en la fuente original.

Y lo que es más importante, vemos también que algunas de las nuevas entradas ``state`` también son nulas, lo que significa que no había ninguna entrada correspondiente en la clave ``abbrevs``.
Averigüemos qué regiones carecen de esta coincidencia:

In [35]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()  #unique devuelve unico por cada valor isnull

array(['PR', 'USA'], dtype=object)

Podemos deducir rápidamente el problema: nuestros datos de población incluyen entradas para Puerto Rico (PR) y los Estados Unidos en su conjunto (USA), mientras que estas entradas no aparecen en la clave de abreviatura del estado.
Podemos solucionarlo rápidamente introduciendo las entradas adecuadas:

In [38]:
pop['state/region'].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR', 'USA'],
      dtype=object)

In [39]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()  #vuelvo a comprobar si hay algún nulo más, si se me pasó alguno

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

Se acabaron los nulos en la columna ``state``: ¡listo!

Ahora podemos fusionar el resultado con los datos del área mediante un procedimiento similar.
Al examinar nuestros resultados, querremos unirlos en la columna ``state`` de ambos:

In [41]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [42]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AK,total,1990,553290.0,Alaska,656425.0
1,AK,under18,1990,177502.0,Alaska,656425.0
2,AK,total,1992,588736.0,Alaska,656425.0
3,AK,under18,1991,182180.0,Alaska,656425.0
4,AK,under18,1992,184878.0,Alaska,656425.0


De nuevo, vamos a comprobar si hay nulos para ver si hay algún desajuste:

In [43]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

Hay nulos en la columna ``area``; podemos echar un vistazo para ver qué regiones se ignoraron aquí:

In [44]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

Vemos que nuestro ``DataFrame`` de ``areas`` no contiene el área de Estados Unidos en su conjunto.
Podríamos insertar el valor apropiado (usando la suma de las áreas de todos los estados, por ejemplo), pero en este caso simplemente eliminaremos los valores nulos porque la densidad de población de todo Estados Unidos no es relevante para nuestra investigación actual:

In [46]:
areas  #vemos que NO tenemos el area completa de USA, podríamos sumar todas las areas de los estados.
     #Pero igual paso de ese valor, y decido que no necesito esa densidad.

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707
5,Colorado,104100
6,Connecticut,5544
7,Delaware,1954
8,Florida,65758
9,Georgia,59441


In [47]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AK,total,1990,553290.0,Alaska,656425.0
1,AK,under18,1990,177502.0,Alaska,656425.0
2,AK,total,1992,588736.0,Alaska,656425.0
3,AK,under18,1991,182180.0,Alaska,656425.0
4,AK,under18,1992,184878.0,Alaska,656425.0


Ahora tenemos todos los datos que necesitamos. Para responder a la pregunta que nos interesa, vamos a seleccionar primero la parte de los datos correspondiente al año 2000 y la población total.
Usaremos la función ``query()`` para hacerlo rápidamente (esto requiere tener instalado el paquete ``numexpr``.

In [51]:
final['year'] == 2010

0       False
1       False
2       False
3       False
4       False
        ...  
2539    False
2540    False
2541    False
2542    False
2543    False
Name: year, Length: 2476, dtype: bool

In [53]:
final [(final['year'] == 2010) & (final['ages'] == 'total')] #con una máscara booleana, filtramos el dataframe


,state/region,ages,year,population,state,area (sq. mi)
43,AK,total,2010,713868.0,Alaska,656425.0
51,AL,total,2010,4785570.0,Alabama,52423.0
141,AR,total,2010,2922280.0,Arkansas,53182.0
149,AZ,total,2010,6408790.0,Arizona,114006.0
197,CA,total,2010,37333601.0,California,163707.0
283,CO,total,2010,5048196.0,Colorado,104100.0
293,CT,total,2010,3579210.0,Connecticut,5544.0
341,DC,total,2010,605125.0,District of Columbia,68.0
427,DE,total,2010,899711.0,Delaware,1954.0
475,FL,total,2010,18846054.0,Florida,65758.0


In [54]:
data2010 = final.query("year == 2010 & ages == 'total'") #comillas dobles a todas la query, y sencillas al valor concreto. 
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
43,AK,total,2010,713868.0,Alaska,656425.0
51,AL,total,2010,4785570.0,Alabama,52423.0
141,AR,total,2010,2922280.0,Arkansas,53182.0
149,AZ,total,2010,6408790.0,Arizona,114006.0
197,CA,total,2010,37333601.0,California,163707.0


Ahora vamos a calcular la densidad de población y mostrarla por orden.
Empezaremos por volver a indexar nuestros datos en el estado y, a continuación, calcularemos el resultado:

In [55]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [58]:
density.head()

state
Alaska          1.087509
Alabama        91.287603
Arkansas       54.948667
Arizona        56.214497
California    228.051342
dtype: float64

In [60]:
density.sort_values(ascending=False, inplace=True) #es lo mismo que    density = density.sort_values(ascending=False) 

In [66]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

El resultado es una clasificación de los estados de EE.UU. más Washington DC, y Puerto Rico por orden de densidad de población en 2010, en residentes por milla cuadrada.
Podemos ver que, con diferencia, la región más densa en este conjunto de datos es Washington DC (es decir, el Distrito de Columbia); entre los estados, el más denso es Nueva Jersey.

También podemos comprobar el final de la lista:

In [69]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

Vemos que el estado menos denso, con diferencia, es Alaska, con una media de poco más de un residente por milla cuadrada.

Este tipo de fusión de datos desordenados es una tarea habitual cuando se intenta responder a preguntas utilizando fuentes de datos del mundo real.

<!--NAVIGATION-->
< [Concatenado](5-Concatenado.ipynb) | [Agregación y agrupación](7-Agregaciones_y_agrupaciones.ipynb) >
